In [ ]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
import re
import numpy as np
import pandas as pd
import gensim
import os
import matplotlib.pyplot as plt
import string
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM, Bidirectional
from tensorflow.keras.layers import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
import emoji

# Define emoji mappings
positive_emojis = ["😊", "😄", "🥳","❤️"]
negative_emojis = ["😢", "😠", "😞"]
neutral_emojis = ["😐", "😑", "😶"]

# Function to detect emojis in text
def detect_emojis(text):
  emojis = []
  for character in text:
    if character in emoji.EMOJI_DATA:
      emojis.append(character)
  return emojis

# Example usage
text = "I love Colgate but I don't like its taste 😊❤️"
detected_emojis = detect_emojis(text)
print("Detected emojis:", detected_emojis)

Detected emojis: ['😊', '❤']


In [ ]:
import os
try:
    device_name = os.environ['COLAB_TPU_ADDR']
    TPU_ADDRESS = 'grpc://' + device_name
    print('Found TPU at: {}'.format(TPU_ADDRESS))
except KeyError:
    print('TPU not found')

Found TPU at: grpc://10.38.174.154:8470


In [ ]:
import pandas as pd

# Read the JSON data using Pandas
data_1 = pd.read_json("/content/Sarcasm_Headlines_Dataset.json", lines=True)
data_2 = pd.read_json("/content/Sarcasm_Headlines_Dataset_v2.json", lines=True)

# Combine the data into a single DataFrame
data = pd.concat([data_1, data_2])

# Preview the first few rows of the data
print(data.head())

                                        article_link  \
0  https://www.huffingtonpost.com/entry/versace-b...   
1  https://www.huffingtonpost.com/entry/roseanne-...   
2  https://local.theonion.com/mom-starting-to-fea...   
3  https://politics.theonion.com/boehner-just-wan...   
4  https://www.huffingtonpost.com/entry/jk-rowlin...   

                                            headline  is_sarcastic  
0  former versace store clerk sues over secret 'b...             0  
1  the 'roseanne' revival catches up to our thorn...             0  
2  mom starting to fear son's web series closest ...             1  
3  boehner just wants wife to listen, not come up...             1  
4  j.k. rowling wishes snape happy birthday in th...             0  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
len(data)

55328

In [ ]:
counts = data['is_sarcastic'].value_counts()
print(counts)


0    29970
1    25358
Name: is_sarcastic, dtype: int64


In [ ]:
df = data

In [ ]:
df = df.drop_duplicates()

In [ ]:
df

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
28497,https://www.theonion.com/tyson-holds-contest-t...,tyson holds contest to let fans submit new ide...,1
28509,https://politics.theonion.com/increasingly-coc...,increasingly cocky bernie sanders announces he...,1
28520,https://www.theonion.com/cash-strapped-zuckerb...,cash-strapped zuckerberg forced to sell 11 mil...,1
28544,https://local.theonion.com/grocery-store-bar-a...,grocery store bar actually has great little ha...,1


In [ ]:
pos_data = df.loc[df['is_sarcastic'] == 1]
nos_data = df.loc[df['is_sarcastic'] == 0]
print("Sarcastic Data-",len(pos_data))
print("Non-Sarcastic Data-",len(nos_data))

Sarcastic Data- 13633
Non-Sarcastic Data- 14984


In [ ]:
def clean_text(text):
    text = text.lower()

    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = pattern.sub('', text)
    text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

    text = emoji.sub(r'', text)
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"did't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"have't", "have not", text)
    text = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-]", "", text)
    return text

In [ ]:
import nltk
nltk.data.path.append('/content/nltk_data')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('words')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def CleanTokenize(df):
    head_lines = list()
    lines = df["headline"].values.tolist()

    for line in lines:
        line = clean_text(line)
        # tokenize the text
        tokens = word_tokenize(line)
        # remove puntuations
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove non alphabetic characters
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        # remove stop words
        words = [w for w in words if not w in stop_words]
        head_lines.append(words)
    return head_lines

head_lines = CleanTokenize(data)

In [ ]:
validation_split = 0.2
max_length = 25


tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(head_lines)
sequences = tokenizer_obj.texts_to_sequences(head_lines)

word_index = tokenizer_obj.word_index
print("unique tokens - ",len(word_index))
vocab_size = len(tokenizer_obj.word_index) + 1
print('vocab size -', vocab_size)

lines_pad = pad_sequences(sequences, maxlen=max_length, padding='post')
sentiment =  data['is_sarcastic'].values

indices = np.arange(lines_pad.shape[0])
np.random.shuffle(indices)
lines_pad = lines_pad[indices]
sentiment = sentiment[indices]

num_validation_samples = int(validation_split * lines_pad.shape[0])

X_train_pad = lines_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = lines_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

unique tokens -  28657
vocab size - 28658


In [ ]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model("/content/model.h5")

In [ ]:
import pandas as pd
def predict_sarcasm(s):
    x_final = pd.DataFrame({"headline":[s]})
    test_lines = CleanTokenize(x_final)
    test_sequences = tokenizer_obj.texts_to_sequences(test_lines)
    test_review_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')
    pred = loaded_model.predict(test_review_pad)
    pred*=100
    if pred[0][0]>=50: return "It's a sarcasm!"
    else: return "It's not a sarcasm."

# Make predictions using the loaded model
prediction = predict_sarcasm("I just won million dollars")
print(prediction)

1/1 [==============================] - 0s 325ms/step
It's a sarcasm!


In [ ]:
predict_sarcasm("I want million dollars.")

1/1 [==============================] - 0s 36ms/step


"It's not a sarcasm."

In [ ]:
import emoji

# Define emoji mappings
positive_emojis = ["😀", "😃", "😄", "😁", "😆", "😅", "😂", "🤣", "😊", "😇", "🙂", "😉", "😌", "😍", "🥰", "😘", "😗", "😙", "😚", "😋", "😛", "😝", "😜", "🤪", "😎", "🤩", "🥳", "😏", "😬", "🤗"]
negative_emojis = ["🥹", "🥲", "☺️", "😐", "😑", "😶", "🙃", "😶‍🌫", "🤔", "🫣", "🤭", "🫡", "🫢", "🫡", "🤫", "🫠", "🤥", "😶", "🫥", "😐", "🫤", "😑", "🫨", "🙄", "😯", "😦", "😧", "😮", "😲", "🥱", "😴", "🤤", "😪", "😵", "🤐", "🥴", "🤢", "🤧", "😷", "🤒", "🤕", "🤑", "🤠"]
neutral_emojis = ["😞", "😔", "😟", "😕", "🙁", "☹️", "😣", "😖", "😫", "😩", "🥺", "😢", "😭", "😤", "😠", "😡", "🤬", "🤯", "😳", "🥵", "🥶", "😱", "😨", "😰", "😥", "😓", "😞", "😧", "😦", "😈", "👿", "👹", "👺", "💩", "😵", "😿"]

# Function to detect emojis in text
def detect_emojis(text):
  emojis = []
  for character in text:
    if character in emoji.EMOJI_DATA:
      emojis.append(character)
  return emojis

# Example usage
text = "I love Colgate but I don't like its taste 😊❤️"
detected_emojis = detect_emojis(text)
print("Detected emojis:", detected_emojis)

Detected emojis: ['😊', '❤']


In [ ]:
def predict_sarcasm_with_emojis(text):
    # Detect emojis in the input text
    detected_emojis = detect_emojis(text)

    # Predict sarcasm
    sarcasm_prediction = predict_sarcasm(text)

    # Classify detected emojis as positive, negative, or neutral
    emoji_sentiment = classify_emojis(detected_emojis)

    # Return sarcasm prediction and emoji sentiment
    return sarcasm_prediction, emoji_sentiment

def detect_emojis(text):
    # Initialize an empty list to store detected emojis
    detected_emojis = []

    # Iterate through each character in the input text
    for character in text:
        # Check if the character is an emoji
        if emoji.is_emoji(character):
            # If it's an emoji, append it to the list of detected emojis
            detected_emojis.append(character)

    # Return the list of detected emojis
    return detected_emojis

def classify_emojis(emojis):
    # Initialize counts for positive, negative, and neutral emojis
    positive_count = 0
    negative_count = 0
    neutral_count = 0

    # Define lists of positive, negative, and neutral emojis
    positive_emojis = ["😀", "😃", "😄", "😁", "😆", "😅", "😂", "🤣", "😊", "😇", "🙂", "😉", "😌", "😍", "🥰", "😘", "😗", "😙", "😚", "😋", "😛", "😝", "😜", "🤪", "😎", "🤩", "🥳", "😏", "😬", "🤗"]
    negative_emojis = ["🥹", "🥲", "☺️", "😐", "😑", "😶", "🙃", "😶‍🌫", "🤔", "🫣", "🤭", "🫡", "🫢", "🫡", "🤫", "🫠", "🤥", "😶", "🫥", "😐", "🫤", "😑", "🫨", "🙄", "😯", "😦", "😧", "😮", "😲", "🥱", "😴", "🤤", "😪", "😵", "🤐", "🥴", "🤢", "🤧", "😷", "🤒", "🤕", "🤑", "🤠"]
    neutral_emojis = ["😞", "😔", "😟", "😕", "🙁", "☹️", "😣", "😖", "😫", "😩", "🥺", "😢", "😭", "😤", "😠", "😡", "🤬", "🤯", "😳", "🥵", "🥶", "😱", "😨", "😰", "😥", "😓", "😞", "😧", "😦", "😈", "👿", "👹", "👺", "💩", "😵", "😿"]

    # Iterate through each emoji in the input list
    for emoji in emojis:
        # Check if the emoji is positive
        if emoji in positive_emojis:
            positive_count += 1
        # Check if the emoji is negative
        elif emoji in negative_emojis:
            negative_count += 1
        # If it's not positive or negative, consider it neutral
        else:
            neutral_count += 1

    # Determine the overall sentiment based on the majority of emojis
    if positive_count > negative_count and positive_count > neutral_count:
        return "Positive"
    elif negative_count > positive_count and negative_count > neutral_count:
        return "Negative"
    else:
        return "Neutral"

# Example usage
text = "I love Colgate but I don't like its taste 🙁"
sarcasm_prediction, emoji_sentiment = predict_sarcasm_with_emojis(text)
print("Sarcasm Prediction:", sarcasm_prediction)
print("Emoji Sentiment:", emoji_sentiment)

1/1 [==============================] - 0s 26ms/step
Sarcasm Prediction: It's not a sarcasm.
Emoji Sentiment: Neutral


In [ ]:
text = "I just won million dollars 😆😶"
sarcasm_prediction, emoji_sentiment = predict_sarcasm_with_emojis(text)
print("Sarcasm Prediction:", sarcasm_prediction)
print("Emoji Sentiment:", emoji_sentiment)

1/1 [==============================] - 0s 25ms/step
Sarcasm Prediction: It's a sarcasm!
Emoji Sentiment: Neutral


In [ ]:
import os

# Print the current working directory
print("Current working directory:", os.getcwd())

# Check if the directory '/kaggle/input' exists
print("Directory exists:", os.path.exists('/kaggle/input'))

# Print files and directories in '/kaggle/input'
for dirname, _, filenames in os.walk('/kaggle/input'):
    print("Directory:", dirname)
    print("Files:", filenames)
    for filename in filenames:
        print(os.path.join(dirname, filename))

Current working directory: /content
Directory exists: True
Directory: /kaggle/input
Files: []


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

wb = gc.open_by_key("1XJ1f1yLEeh0igyXiim6UEiPjTPDeNja59TweDBLAVjA")

# Get the first worksheet (index 0)
ws = wb.get_worksheet(0)

# Check if the worksheet was successfully retrieved
if ws:
    print("Worksheet retrieved successfully.")
    # get_all_values gives a list of rows.
    rows = ws.get_all_values()
    print(rows)
else:
    print("Worksheet not found.")


Worksheet retrieved successfully.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
df1 = pd.DataFrame(rows)

# Print the DataFrame to inspect its structure
print(df1)

                                            0   \
0                                      postUrl   
1     https://www.instagram.com/p/CotbxWksI1k/   
2     https://www.instagram.com/p/CaU0zQ9lk4Q/   
3     https://www.instagram.com/p/CosKMGtuzza/   
4     https://www.instagram.com/p/CnnDbSnuwJU/   
...                                        ...   
5213  https://www.instagram.com/p/CJaUauDKN_g/   
5214  https://www.instagram.com/p/CJWkp-ipIAB/   
5215  https://www.instagram.com/p/CJVakchFy7y/   
5216  https://www.instagram.com/p/CJQveYrpYKB/   
5217  https://www.instagram.com/p/CJQu_-vJzJq/   

                                                1                   2   \
0                                       profileUrl            username   
1        https://www.instagram.com/indigirlreviews     indigirlreviews   
2         https://www.instagram.com/tiffanyguanuna      tiffanyguanuna   
3        https://www.instagram.com/sherellroweshow     sherellroweshow   
4               https://www.i

In [ ]:
cs = pd.read_csv("/content/Colgate Scrapped Dataset - Post_data_Final.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/Colgate Scrapped Dataset - Post_data_Final.csv'

In [ ]:
cs

In [ ]:
rows = cs.values.tolist()

In [ ]:
#defining positive, negative and connectors lists of words

positive = ["love","like","happy","good","excellent","great","nice","wonderful","fantastic","amazing","awesome","terrific","fabulous","delightful","pleasure","enjoy",
    "satisfy","smile","laugh","success","kind","generous","gracious","forgiving","compassionate","understanding","trust","loyal","honest","fair","brave",
    "adventurous","confident","creative","smart","intelligent","insightful","inspiring","motivated","passionate","productive","focused","disciplined","healthy",
    "energetic","athletic","beautiful","attractive","fashionable","stylish","successful","wealthy","rich","prosperous","abundant","blessed","fortunate","lucky",
    "grateful","content","peaceful","calm","relaxed","tranquil","serene","happy","joyful","blissful","ecstatic","hopeful","optimistic","courageous","confident",
    "proud","admire","respect","appreciate","significant","glad","perfect","rapturous", "fascinating", "enriching", "tranquilizing", "blissful", "serendipitous", "inspirational", "pleasurable", "upbeat", "ecstatic",
    "delicious", "exhilarating", "glamorous","Colgate" ,"majestic", "stunning", "irresistible", "breathtaking", "prodigious", "miraculous", "miracle","thrilling",
    "mind-blowing", "exquisite", "alluring", "extraordinary", "spectacular", "outstanding", "remarkable", "unforgettable", "marvelous", "unbeatable", "brilliant", "genius", "stellar", "fantabulous", "incredible", "sublime",
    "superb", "terrific", "top-notch", "wondrous","Colgate"]

negative = ["not","neither","don't","didn't","wouldn't","shouldn't","haven't","hadn't","can't","hate","unhappy","bad","terrible","uncomfortable","horrible","awful","disgusting","ugly","painful",
    "sorrow","hurt","upset","frustrated","angry","annoyed","disappointed","regret","guilt","shame","fear","scared","anxious","worried","nervous","doubt","insecure",
    "jealous","envy","betray","abandon","alone","lonely","depressed","sad","miserable","gloomy","hopeless","despair","powerless","worthless","rejected","humiliated",
    "insulted","offended","disrespected","unappreciated","frustrated","overwhelmed","exhausted","tired","bored","stressed","irritated","annoying","negative","critical",
    "mean","cruel","selfish","arrogant","ignorant","stupid","lazy","unmotivated","unproductive","unsuccessful","poor","struggle","failure","loss","difficult","tragic",
    "disastrous","devastating","pathetic","disappointing","regretful","heartbroken","wounded","insensitive","impolite","unfriendly","cynical","pessimistic","cowardly",
    "indecisive","irresponsible","foolish","weak","worthless","incompetent","dependent","unreliable","untrustworthy","unhealthy","unfit","unattractive","poor","broke",
    "struggling","difficult","disgusted","offended","trapped","stuck","lonely","empty","lost","confused","ashamed","humiliated","horrified","disillusioned","depressed",
    "unhappy","bitter","enraged","resentful","discontented","displeased","worried","nervous","fearful","anxious","terrified","petrified","timid","shy","insecure",
    "uncertain","doubtful","skeptical","disbelieving","inattentive","distracted","absent-minded","forgetful","neglectful","careless","reckless","irresponsible",
    "impulsive","careless","inconsiderate","thoughtless","selfish","self-centered","egotistical","arrogant","ignorant","inflexible","stubborn","rigid","boring","weird"
    "monotonous","repetitive","tedious","dull","uninteresting","unexciting","mundane","trivial","unimportant","insignificant","meaningless","worthless","disappointed"]

#connectors are the subordinating conjunctions which join two or more phrases + punctuations
connectors = ["for","and","nor","but","or","yet","so",",",";"]

In [ ]:
def encodeSentence(sentence):
  # words = sentence.split()
  words = re.findall(r"[\w']+|[^\w\s]", sentence)
  encoded_sentence = []
  for word in words:
    if any(char.isdigit() or char.isalpha() for char in word):
      if word in positive:
        encoded_sentence.append(2)

      elif word in negative:
        encoded_sentence.append(0)

      elif word in connectors:
        encoded_sentence.append(3)

      else:
        encoded_sentence.append(1)

    else:
      encoded_sentence.append(3)

  return encoded_sentence

In [ ]:
def createTree(encoded_sentence):
  tree_diagram =[]
  N = len(encoded_sentence)
  count = 0

  for i in range(0,N+1):
    tree_diagram.append([])

  for code in encoded_sentence:
    if count == N:
      tree_diagram[count-1].append(code)
    elif count == 0:
      tree_diagram[count].append(code)
      tree_diagram[count].append(None)
    else:
      tree_diagram[count].append(code)
      tree_diagram[count].append(None)

    count = count+1

  return tree_diagram

In [ ]:
def calculateValue(temp,tree):
  result = 1

  if tree[temp][0]==0 and tree[temp][1]==1:
        result=0
  elif tree[temp][0] == 0 and tree[temp][1] == 0:
        result = 2
  elif tree[temp][0] == 1 and tree[temp][1] == 1:
        result = 1
  elif tree[temp][0] == 1 and tree[temp][1] == 0:
        result = 0
  elif tree[temp][0] == 1 and tree[temp][1] == 2:
        result = 2
  elif tree[temp][0] == 2 and tree[temp][1] == 2:
        result = 2
  elif tree[temp][0] == 2 and tree[temp][1] == 1:
      result = 2
  elif tree[temp][0] == 0 and tree[temp][1] == 2:
        result = 0
  elif tree[temp][0] == 2 and tree[temp][1] == 0:
        result = 0

  return result

In [ ]:
def calculateSentiment(temp):
  result = 1

  if sentiment_tree[temp][0]==0 and sentiment_tree[temp][1]==1:
        result=0
  elif sentiment_tree[temp][0] == 0 and sentiment_tree[temp][1] == 0:
        result = 0
  elif sentiment_tree[temp][0] == 1 and sentiment_tree[temp][1] == 1:
        result = 1
  elif sentiment_tree[temp][0] == 1 and sentiment_tree[temp][1] == 0:
        result = 0
  elif sentiment_tree[temp][0] == 1 and sentiment_tree[temp][1] == 2:
        result = 2
  elif sentiment_tree[temp][0] == 2 and sentiment_tree[temp][1] == 2:
        result = 2
  elif sentiment_tree[temp][0] == 2 and sentiment_tree[temp][1] == 1:
      result = 2
  elif sentiment_tree[temp][0] == 0 and sentiment_tree[temp][1] == 2:
        result = 2
  elif sentiment_tree[temp][0] == 2 and sentiment_tree[temp][1] == 0:
        result = 0

  return result

In [ ]:
#Live Prediction
def prediction(s):
    x_final = pd.DataFrame({"headline":[s]})
    test_lines = CleanTokenize(x_final)
    test_sequences = tokenizer_obj.texts_to_sequences(test_lines)
    test_review_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')
    pred = loaded_model.predict(test_review_pad)
    pred*=100
    if pred[0][0]>=50: return "It's a sarcasm!"
    else: return "It's not a sarcasm."

In [ ]:
prediction("I was depressed. He asked me to be happy. I am not depressed anymore")

1/1 [==============================] - 0s 25ms/step


"It's a sarcasm!"

In [ ]:
import re

result_list = ['Negative', 'Neutral', 'Positive']
final = []

testSentences = ["I love Colgate but I don't like its taste"]

for sentence in testSentences:
  # sentence = row['description']
  tree = createTree(encodeSentence(sentence))
  sentiment = []
  result = []
  encoded_sentence = encodeSentence(sentence)
  # print(sentence, encoded_sentence)
  list_splits = []
  start_index = 0
  #splitting the sentences by connectors
  for i, value in enumerate(encoded_sentence):
      if value == 3:
          list_splits.append(encoded_sentence[start_index:i])
          start_index = i + 1
  list_splits.append(encoded_sentence[start_index:])
  # print(encoded_sentence, list_splits)
  # Create a tree for each split
  for split in list_splits:
    if split:
      tree = createTree(split)
      # print(split, tree)
      #getting sentiment of splits
      N = len(split)
      temp = N
      # print(N)
      for i in range(0, N):
        temp = temp -1
        if i == 0:
          tree[temp][1]=1
        else:
          tree[temp][1] = calculateValue(temp+1,tree)
    sentiment.append(calculateValue(temp,tree))
    # print(tree, sentiment)

#       # print("Encoded Sentence:", encoded_sentence)
#       # print("Split Lists:", list_splits)
#       # print("Sentiment:", sentiment)

  #creating tree for sentiments
  sentiment_tree = createTree(sentiment)
  # print(sentiment, sentiment_tree)
  # print(len(sentiment_tree))
  N = len(sentiment)
  # print(N)
  temp = N

  for i in range(0,N):
    # print(sentiment_tree)
    temp=temp-1
    # print(temp, i)
    if i == 0:
      sentiment_tree[temp][1]=1
    else:
      sentiment_tree[temp][1] = calculateSentiment(temp+1)
  # print(sentiment, sentiment_tree)

  final.append(calculateSentiment(temp))

In [ ]:
for i in final:
  if i == 0:
    print(f'{testSentences[0]} - Negative')
  if i == 1:
    print(f'{testSentences[0]} - Neutral')
  if i == 2:
    print(f'{testSentences[0]} - Positive')

I love Colgate but I don't like its taste - Negative


In [ ]:
text = "I love Colgate but I don't like its taste 🙁"
sarcasm_prediction, emoji_sentiment = predict_sarcasm_with_emojis(text)
print(sarcasm_prediction)
print(emoji_sentiment)

1/1 [==============================] - 0s 24ms/step
It's not a sarcasm.
Neutral


In [ ]:
def perform_sentiment_analysis(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation marks
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize the text into words
    words = text.split()

    # Initialize sentiment score
    score = 0

    # Iterate through each word in the text
    for word in words:
        # Check if the word is in the positive list
        if word in positive:
            score += 1
        # Check if the word is in the negative list
        elif word in negative:
            score -= 1

    # Classify sentiment based on the score
    if score > 0:
        return "Positive"
    elif score < 0:
        return "Negative"
    else:
        return "Neutral"

In [ ]:
text = "I love Colgate but I don't like its taste 🙁"
sarcasm_prediction, emoji_sentiment = predict_sarcasm_with_emojis(text)
sentiment = perform_sentiment_analysis(text)

print("Sarcasm Prediction:", sarcasm_prediction)
print("Emoji Sentiment:", emoji_sentiment)
print("Sentiment Analysis:", sentiment)

1/1 [==============================] - 0s 25ms/step
Sarcasm Prediction: It's not a sarcasm.
Emoji Sentiment: Neutral
Sentiment Analysis: Positive


In [ ]:
import re

def predict_sentiment_with_emoji_and_sarcasm(text):
    # Sentiment Tree Prediction
    def encodeSentence(sentence):
        # words = sentence.split()
        words = re.findall(r"[\w']+|[^\w\s]", sentence)
        encoded_sentence = []
        for word in words:
            if any(char.isdigit() or char.isalpha() for char in word):
                if word in positive:
                    encoded_sentence.append(2)
                elif word in negative:
                    encoded_sentence.append(0)
                elif word in connectors:
                    encoded_sentence.append(3)
                else:
                    encoded_sentence.append(1)
            else:
                encoded_sentence.append(3)
        return encoded_sentence

    def createTree(encoded_sentence):
        tree_diagram = []
        N = len(encoded_sentence)
        count = 0

        for i in range(0, N + 1):
            tree_diagram.append([])

        for code in encoded_sentence:
            if count == N:
                tree_diagram[count - 1].append(code)
            elif count == 0:
                tree_diagram[count].append(code)
                tree_diagram[count].append(None)
            else:
                tree_diagram[count].append(code)
                tree_diagram[count].append(None)

            count = count + 1

        return tree_diagram

    def calculateValue(temp, tree):
        result = 1
        if tree[temp][0] == 0 and tree[temp][1] == 1:
            result = 0
        elif tree[temp][0] == 0 and tree[temp][1] == 0:
            result = 2
        elif tree[temp][0] == 1 and tree[temp][1] == 1:
            result = 1
        elif tree[temp][0] == 1 and tree[temp][1] == 0:
            result = 0
        elif tree[temp][0] == 1 and tree[temp][1] == 2:
            result = 2
        elif tree[temp][0] == 2 and tree[temp][1] == 2:
            result = 2
        elif tree[temp][0] == 2 and tree[temp][1] == 1:
            result = 2
        elif tree[temp][0] == 0 and tree[temp][1] == 2:
            result = 0
        elif tree[temp][0] == 2 and tree[temp][1] == 0:
            result = 0
        return result

    def calculateSentiment(temp, sentiment_tree):
        result = 1
        if sentiment_tree[temp][0] == 0 and sentiment_tree[temp][1] == 1:
            result = 0
        elif sentiment_tree[temp][0] == 0 and sentiment_tree[temp][1] == 0:
            result = 2
        elif sentiment_tree[temp][0] == 1 and sentiment_tree[temp][1] == 1:
            result = 1
        elif sentiment_tree[temp][0] == 1 and sentiment_tree[temp][1] == 0:
            result = 0
        elif sentiment_tree[temp][0] == 1 and sentiment_tree[temp][1] == 2:
            result = 2
        elif sentiment_tree[temp][0] == 2 and sentiment_tree[temp][1] == 2:
            result = 2
        elif sentiment_tree[temp][0] == 2 and sentiment_tree[temp][1] == 1:
            result = 2
        elif sentiment_tree[temp][0] == 0 and sentiment_tree[temp][1] == 2:
            result = 0
        elif sentiment_tree[temp][0] == 2 and sentiment_tree[temp][1] == 0:
            result = 0
        return result

    def sarcasm_prediction(text):
        x_final = pd.DataFrame({"headline":[text]})
        test_lines = CleanTokenize(x_final)
        test_sequences = tokenizer_obj.texts_to_sequences(test_lines)
        test_review_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')
        pred = loaded_model.predict(test_review_pad)
        pred *= 100
        if pred[0][0] >= 50:
            return "Sarcasm Detected!"
        else:
            return "Not a Sarcasm."

    def emoji_sentiment(text):
        detected_emojis = detect_emojis(text)
        emoji_sentiment = classify_emojis(detected_emojis)
        return emoji_sentiment

    result_list = ['Negative', 'Neutral', 'Positive']
    final_sentiment = None

    testSentences = [text]

    for sentence in testSentences:
        tree = createTree(encodeSentence(sentence))
        sentiment = []
        result = []
        encoded_sentence = encodeSentence(sentence)

        list_splits = []
        start_index = 0

        for i, value in enumerate(encoded_sentence):
            if value == 3:
                list_splits.append(encoded_sentence[start_index:i])
                start_index = i + 1
        list_splits.append(encoded_sentence[start_index:])

        for split in list_splits:
            if split:
                tree = createTree(split)
                N = len(split)
                temp = N
                for i in range(0, N):
                    temp = temp - 1
                    if i == 0:
                        tree[temp][1] = 1
                    else:
                        tree[temp][1] = calculateValue(temp + 1, tree)
                sentiment.append(calculateValue(temp, tree))

        sentiment_tree = createTree(sentiment)
        N = len(sentiment)
        temp = N

        for i in range(0, N):
            temp = temp - 1
            if i == 0:
                sentiment_tree[temp][1] = 1
            else:
                sentiment_tree[temp][1] = calculateSentiment(temp + 1, sentiment_tree)

        final_sentiment = calculateSentiment(temp, sentiment_tree)

    sarcasm_result = sarcasm_prediction(text)
    emoji_result = emoji_sentiment(text)

    return final_sentiment, sarcasm_result, emoji_result

# Test the function with a sample text
text = "I love Colgate but I don't like its taste"
final_sentiment, sarcasm_result, emoji_result = predict_sentiment_with_emoji_and_sarcasm(text)
print("Final Sentiment:", result_list[final_sentiment])
print("Sarcasm Prediction:", sarcasm_result)
print("Emoji Sentiment:", emoji_result)


1/1 [==============================] - 0s 23ms/step
Final Sentiment: Negative
Sarcasm Prediction: Not a Sarcasm.
Emoji Sentiment: Neutral


In [ ]:
text = "I love Colgate but I don't like its taste 😑"
final_sentiment, sarcasm_result, emoji_result = predict_sentiment_with_emoji_and_sarcasm(text)
print("Final Sentiment:", result_list[final_sentiment])
print("Sarcasm Prediction:", sarcasm_result)
print("Emoji Sentiment:", emoji_result)

1/1 [==============================] - 0s 27ms/step
Final Sentiment: Negative
Sarcasm Prediction: Not a Sarcasm.
Emoji Sentiment: Negative


In [ ]:
def check_punctuation(sentence):
    punctuation_marks = ['.', ',', ';', ':', '?', '!', '-', "'", '"', '(', ')', '[', ']', '{', '}', '/', '\\', '<', '>', '|', '*', '&', '@', '#', '%', '^', '_', '+', '=', '~', '`', '$', '&', '£', '€', '¥', '¢', '¤', '§', '©', '®', '™', '¬', '¦', 'µ', '±', '×', '÷', '¢', '฿', '₡', '₢', '₣', '₤', '₥', '₦', '₧', '₨', '₩', '₪', '₫', '€', '₭', '₮', '₯', '₰', '₱', '₲', '₳', '₴', '₵', '₶', '₷', '₸', '₹', '₺', '₻', '₼', '₽', '₾', '₿']

    punctuation_meanings = {
        '.': 'period (indicates the end of a sentence)',
        ',': 'comma (separates items in a list or series)',
        ';': 'semicolon (indicates a pause or separates independent clauses)',
        ':': 'colon (introduces a list or explanation)',
        '?': 'question mark (indicates a question)',
        '!': 'exclamation mark (indicates strong feeling or surprise)',
        '-': 'hyphen (joins words or parts of words)',
        "'": 'apostrophe (indicates possession or omitted letters)',
        '"': 'quotation mark (indicates speech or a quotation)',
        '(': 'left parenthesis (encloses additional information)',
        ')': 'right parenthesis (encloses additional information)',
        '[': 'left square bracket (encloses additional information)',
        ']': 'right square bracket (encloses additional information)',
        '{': 'left curly bracket (encloses additional information)',
        '}': 'right curly bracket (encloses additional information)',
        '/': 'forward slash (used in URLs, paths, or fractions)',
        '\\': 'backslash (used in file paths or escape characters)',
        '<': 'less than sign (used in mathematical expressions or HTML tags)',
        '>': 'greater than sign (used in mathematical expressions or HTML tags)',
        '|': 'vertical bar or pipe (used in programming or to denote alternatives)',
        '*': 'asterisk (used for multiplication, wildcard characters, or emphasis)',
        '&': 'ampersand (represents "and", used in HTML or programming)',
        '@': 'at symbol (used in email addresses or social media handles)',
        '#': 'hashtag (used in social media to categorize topics)',
        '%': 'percent sign (indicates a percentage)',
        '^': 'caret (used in mathematical expressions or to denote exponentiation)',
        '_': 'underscore (used in programming or for emphasis)',
        '+': 'plus sign (used in mathematical expressions or to denote addition)',
        '=': 'equal sign (indicates equality or assignment)',
        '~': 'tilde (used in programming or as a diacritical mark)',
        '`': 'grave accent (used in programming or as a diacritical mark)',
        '$': 'dollar sign (used to denote currency)',
        '£': 'pound sign (used to denote currency)',
        '€': 'euro sign (used to denote currency)',
        '¥': 'yen sign (used to denote currency)',
        '¢': 'cent sign (used to denote currency)',
        '¤': 'currency sign (used to denote currency)',
        '§': 'section sign (used in legal documents or to denote a section)',
        '©': 'copyright symbol (indicates ownership of intellectual property)',
        '®': 'registered trademark symbol (indicates a registered trademark)',
        '™': 'trademark symbol (indicates a trademark)',
        '¬': 'negation symbol (used in logic or to indicate a negative)',
        '¦': 'broken vertical bar (used to denote alternatives or separate clauses)',
        'µ': 'micro sign (used in scientific notation or to denote micro- units)',
        '±': 'plus-minus sign (indicates a range of values)',
        '×': 'multiplication sign (indicates multiplication)',
        '÷': 'division sign (indicates division)',
        '฿': 'baht sign (used to denote currency)',
        '₡': 'colon sign (used to denote currency)',
        '₢': 'cruzeiro sign (used to denote currency)',
        '₣': 'french franc sign (used to denote currency)',
        '₤': 'lira sign (used to denote currency)',
        '₥': 'mill sign (used to denote currency)',
        '₦': 'naira sign (used to denote currency)',
        '₧': 'peseta sign (used to denote currency)',
        '₨': 'rupee sign (used to denote currency)',
        '₩': 'won sign (used to denote currency)',
        '₪': 'new sheqel sign (used to denote currency)',
        '₫': 'dong sign (used to denote currency)',
        '₭': 'kip sign (used to denote currency)',
        '₮': 'tugrik sign (used to denote currency)',
        '₯': 'drachma sign (used to denote currency)',
        '₰': 'german penny sign (used to denote currency)',
        '₱': 'peso sign (used to denote currency)',
        '₲': 'guarani sign (used to denote currency)',
        '₳': 'austral sign (used to denote currency)',
        '₴': 'hryvnia sign (used to denote currency)',
        '₵': 'cedi sign (used to denote currency)',
        '₶': 'livre tournois sign (used to denote currency)',
        '₷': 'spesmilo sign (used to denote currency)',
        '₸': 'tenge sign (used to denote currency)',
        '₹': 'indian rupee sign (used to denote currency)',
        '₺': 'turkish lira sign (used to denote currency)',
        '₻': 'nordic mark sign (used to denote currency)',
        '₼': 'manat sign (used to denote currency)',
        '₽': 'ruble sign (used to denote currency)',
        '₾': 'lari sign (used to denote currency)',
        '₿': 'bitcoin sign (used to denote cryptocurrency)'
    }


    found_punctuation = []

    for char in sentence:
        if char in punctuation_marks:
            found_punctuation.append((char, punctuation_meanings[char]))

    if not found_punctuation:
        print("No punctuation marks found in the sentence.")
    else:
        print("Punctuation marks found in the sentence:")
        for punctuation, meaning in found_punctuation:
            print(f"{punctuation}: {meaning}")

        print("Overall meaning of the sentence:")
        print(f"The last punctuation mark '{found_punctuation[-1][0]}' suggests '{found_punctuation[-1][1]}'.")


# Example usage:
sentence = "What a beautiful day! Isn't it?"
check_punctuation(sentence)

Punctuation marks found in the sentence:
!: exclamation mark (indicates strong feeling or surprise)
': apostrophe (indicates possession or omitted letters)
?: question mark (indicates a question)
Overall meaning of the sentence:
The last punctuation mark '?' suggests 'question mark (indicates a question)'.


In [ ]:
text = "I love Colgate but I don't like its taste 😑!"
final_sentiment, sarcasm_result, emoji_result = predict_sentiment_with_emoji_and_sarcasm(text)
print("punctuation: ",check_punctuation(text))
print("Final Sentiment:", result_list[final_sentiment])
print("Sarcasm Prediction:", sarcasm_result)
print("Emoji Sentiment:", emoji_result)

1/1 [==============================] - 0s 47ms/step
Punctuation marks found in the sentence:
': apostrophe (indicates possession or omitted letters)
!: exclamation mark (indicates strong feeling or surprise)
Overall meaning of the sentence:
The last punctuation mark '!' suggests 'exclamation mark (indicates strong feeling or surprise)'.
punctuation:  None
Final Sentiment: Negative
Sarcasm Prediction: Not a Sarcasm.
Emoji Sentiment: Negative


In [ ]:
text = "What a beautiful day! Isn't it🥲?"
final_sentiment, sarcasm_result, emoji_result = predict_sentiment_with_emoji_and_sarcasm(text)
print(check_punctuation(text))
print("Final Sentiment:", result_list[final_sentiment])
print("Sarcasm Prediction:", sarcasm_result)
print("Emoji Sentiment:", emoji_result)

1/1 [==============================] - 0s 36ms/step
Punctuation marks found in the sentence:
!: exclamation mark (indicates strong feeling or surprise)
': apostrophe (indicates possession or omitted letters)
?: question mark (indicates a question)
Overall meaning of the sentence:
The last punctuation mark '?' suggests 'question mark (indicates a question)'.
None
Final Sentiment: Positive
Sarcasm Prediction: Not a Sarcasm.
Emoji Sentiment: Negative


In [ ]:
text = "You just broke my car window. Great job!!😆"
final_sentiment, sarcasm_result, emoji_result = predict_sentiment_with_emoji_and_sarcasm(text)
print(check_punctuation(text))
print("Final Sentiment:", result_list[final_sentiment])
print("Sarcasm Prediction:", sarcasm_result)
print("Emoji Sentiment:", emoji_result)

1/1 [==============================] - 0s 39ms/step
Punctuation marks found in the sentence:
.: period (indicates the end of a sentence)
!: exclamation mark (indicates strong feeling or surprise)
!: exclamation mark (indicates strong feeling or surprise)
Overall meaning of the sentence:
The last punctuation mark '!' suggests 'exclamation mark (indicates strong feeling or surprise)'.
None
Final Sentiment: Negative
Sarcasm Prediction: Sarcasm Detected!
Emoji Sentiment: Positive


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

# Provided dataset
punctuation_marks = ['.', ',', ';', ':', '?', '!', '-', "'", '"', '(', ')', '[', ']', '{', '}', '/', '\\', '<', '>', '|', '*', '&', '@', '#', '%', '^', '_', '+', '=', '~', '`', '$', '&', '£', '€', '¥', '¢', '¤', '§', '©', '®', '™', '¬', '¦', 'µ', '±', '×', '÷', '¢', '฿', '₡', '₢', '₣', '₤', '₥', '₦', '₧', '₨', '₩', '₪', '₫', '€', '₭', '₮', '₯', '₰', '₱', '₲', '₳', '₴', '₵', '₶', '₷', '₸', '₹', '₺', '₻', '₼', '₽', '₾', '₿']
punctuation_meanings = {
    '.': 'period (indicates the end of a sentence)',
    ',': 'comma (separates items in a list or series)',
    ';': 'semicolon (indicates a pause or separates independent clauses)',
    ':': 'colon (introduces a list or explanation)',
    '?': 'question mark (indicates a question)',
    '!': 'exclamation mark (indicates strong feeling or surprise)',
    '-': 'hyphen (joins words or parts of words)',
    "'": 'apostrophe (indicates possession or omitted letters)',
    '"': 'quotation mark (indicates speech or a quotation)',
    '(': 'left parenthesis (encloses additional information)',
    ')': 'right parenthesis (encloses additional information)',
    '[': 'left square bracket (encloses additional information)',
    ']': 'right square bracket (encloses additional information)',
    '{': 'left curly bracket (encloses additional information)',
    '}': 'right curly bracket (encloses additional information)',
    '/': 'forward slash (used in URLs, paths, or fractions)',
    '\\': 'backslash (used in file paths or escape characters)',
    '<': 'less than sign (used in mathematical expressions or HTML tags)',
    '>': 'greater than sign (used in mathematical expressions or HTML tags)',
    '|': 'vertical bar or pipe (used in programming or to denote alternatives)',
    '*': 'asterisk (used for multiplication, wildcard characters, or emphasis)',
    '&': 'ampersand (represents "and", used in HTML or programming)',
    '@': 'at symbol (used in email addresses or social media handles)',
    '#': 'hashtag (used in social media to categorize topics)',
    '%': 'percent sign (indicates a percentage)',
    '^': 'caret (used in mathematical expressions or to denote exponentiation)',
    '_': 'underscore (used in programming or for emphasis)',
    '+': 'plus sign (used in mathematical expressions or to denote addition)',
    '=': 'equal sign (indicates equality or assignment)',
    '~': 'tilde (used in programming or as a diacritical mark)',
    '`': 'grave accent (used in programming or as a diacritical mark)',
    '$': 'dollar sign (used to denote currency)',
    '£': 'pound sign (used to denote currency)',
    '€': 'euro sign (used to denote currency)',
    '¥': 'yen sign (used to denote currency)',
    '¢': 'cent sign (used to denote currency)',
    '¤': 'currency sign (used to denote currency)',
    '§': 'section sign (used in legal documents or to denote a section)',
    '©': 'copyright symbol (indicates ownership of intellectual property)',
    '®': 'registered trademark symbol (indicates a registered trademark)',
    '™': 'trademark symbol (indicates a trademark)',
    '¬': 'negation symbol (used in logic or to indicate a negative)',
    '¦': 'broken vertical bar (used to denote alternatives or separate clauses)',
    'µ': 'micro sign (used in scientific notation or to denote micro- units)',
    '±': 'plus-minus sign (indicates a range of values)',
    '×': 'multiplication sign (indicates multiplication)',
    '÷': 'division sign (indicates division)',
    '฿': 'baht sign (used to denote currency)',
    '₡': 'colon sign (used to denote currency)',
    '₢': 'cruzeiro sign (used to denote currency)',
    '₣': 'french franc sign (used to denote currency)',
    '₤': 'lira sign (used to denote currency)',
    '₥': 'mill sign (used to denote currency)',
    '₦': 'naira sign (used to denote currency)',
    '₧': 'peseta sign (used to denote currency)',
    '₨': 'rupee sign (used to denote currency)',
    '₩': 'won sign (used to denote currency)',
    '₪': 'new sheqel sign (used to denote currency)',
    '₫': 'dong sign (used to denote currency)',
    '₭': 'kip sign (used to denote currency)',
    '₮': 'tugrik sign (used to denote currency)',
    '₯': 'drachma sign (used to denote currency)',
    '₰': 'german penny sign (used to denote currency)',
    '₱': 'peso sign (used to denote currency)',
    '₲': 'guarani sign (used to denote currency)',
    '₳': 'austral sign (used to denote currency)',
    '₴': 'hryvnia sign (used to denote currency)',
    '₵': 'cedi sign (used to denote currency)',
    '₶': 'livre tournois sign (used to denote currency)',
    '₷': 'spesmilo sign (used to denote currency)',
    '₸': 'tenge sign (used to denote currency)',
    '₹': 'indian rupee sign (used to denote currency)',
    '₺': 'turkish lira sign (used to denote currency)',
    '₻': 'nordic mark sign (used to denote currency)',
    '₼': 'manat sign (used to denote currency)',
    '₽': 'ruble sign (used to denote currency)',
    '₾': 'lari sign (used to denote currency)',
    '₿': 'bitcoin sign (used to denote cryptocurrency)'
}

# Convert the dataset into a format suitable for training
X = np.array(punctuation_marks).reshape(-1, 1)
y = np.array([punctuation_meanings[p] for p in punctuation_marks])

# Convert punctuation marks to numerical labels
label_encoder = OneHotEncoder(sparse=False)
X_train = label_encoder.fit_transform(X)

# Model training
classifier = LogisticRegression()
classifier.fit(X_train, y)

# Example usage within check_punctuation_ml function
def check_punctuation_ml(punctuation):
    # Encode the punctuation mark using the trained label encoder
    encoded_punctuation = label_encoder.transform([[punctuation]])

    # Predict the punctuation mark meaning using the trained classifier
    predicted_meaning = classifier.predict(encoded_punctuation)[0]

    # Print the results
    print("Punctuation Mark:", punctuation)
    print("Predicted Meaning:", predicted_meaning)

# Example usage
punctuation = ","
check_punctuation_ml(punctuation)